# HTTPS Endpoint로 Model Serving

real-time과 batch predictions Endpoint를 모두 활용할 수 있으며, 이 HOL에서는 Real-time에 대해서만 설명합니다. 
사용 중이신 애플리케이션의 대기시간이 짧은 경우 Model은 real-time API로 배포하며 HTTPS를 통해 단일 예측 요청에 대해 빠르게 예측을 제공합니다.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import time

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

### 이전 Notebook에서 Endpoints를 생성하신 경우에는 Endpoints를 삭제하신 후 이 노트북을 생성하시기 바랍니다. 그렇지 않은 경우 HOL를 수행하시는 동안에 ResourceLimitExceeded error를 보실 수 있습니다.


In [ ]:
list_endpoints = sm.list_endpoints()

for ep in list_endpoints['Endpoints']:
    sm.delete_endpoint(EndpointName=ep['EndpointName'])
    

NextToken = 'None'
while NextToken !='':
    lec = sm.list_endpoint_configs(NextToken=NextToken) if NextToken != 'None' else sm.list_endpoint_configs()
    for epc in lec['EndpointConfigs']:
        print(epc['EndpointConfigName'])
        sm.delete_endpoint_config(EndpointConfigName=epc['EndpointConfigName'])
        time.sleep(3)
    NextToken = lec['NextToken'] if lec.get('NextToken') else ''

NextToken = 'None'
while NextToken !='':
    lec = sm.list_models(NextToken=NextToken) if NextToken != 'None' else sm.list_models()
    for epc in lec['Models']:
        print(epc['ModelName'])
        sm.delete_model(ModelName=epc['ModelName'])
        time.sleep(3)
    NextToken = lec['NextToken'] if lec.get('NextToken') else ''

In [ ]:
%store -r

In [ ]:
print(training_job_name)

# Notebook 내 Model 복사

In [ ]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

In [ ]:
!tar -xvzf ./model.tar.gz

In [ ]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

# Model 배포

단일 Model에 대해 기본적으로 `EndpointConfig`를 생성합니다. 다음 노트북에서 canary 배포와 A/B testing을 지원하는 `EndpointConfig` 전략을 수행할 계획입니다. 

_참고: 리전에 따라 기본적으로 제공되는 다양한 버전/Deep leanring 프레임워크 별로 deep learning 이미지를 제공하고 있으며, 필요에 따라 해당 image를 받아서 추가해서 custom 이미지를 생성할 수 있습니다._

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html


또한, SageMaker에서 학습한 모델이 아닌 경우에도 XXXXX.tar.gz로 압축하여 예측에 활용할 수 있습니다.

In [ ]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

In [ ]:
deployed_model = model.deploy(initial_instance_count=1, # Should use >=2 for high(er) availability 
                              instance_type='ml.m5.large',
#                              accelerator_type='ml.eia2.medium',
                              wait=False)

endpoint_name = deployed_model.endpoint

print('Endpoint name:  {}'.format(endpoint_name))

# Experiment 내 배포 추적

In [ ]:
print(experiment_name)

In [ ]:
print(trial_name)

In [ ]:
import time
from smexperiments.trial import Trial

timestamp = '{}'.format(int(time.time()))

trial = Trial.load(trial_name=trial_name)
print(trial)

In [ ]:
from smexperiments.tracker import Tracker

tracker_deploy = Tracker.create(display_name='deploy', 
                                sagemaker_boto_client=sm)

deploy_trial_component_name = tracker_deploy.trial_component.trial_component_name
print('Deploy trial component name {}'.format(deploy_trial_component_name))

### Trial에 Component로 `deploy` Trial Component 와 Tracker를 추가합니다.

In [ ]:
trial.add_trial_component(tracker_deploy.trial_component)

# Endpoint Name 추적

In [ ]:
tracker_deploy.log_parameters({
    'endpoint_name': endpoint_name,
})

# must save after logging
tracker_deploy.trial_component.save()

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

<h2><span style="color:red">위 Endpoint가 Deploy되기 전까지 기다려 주시기 바랍니다.</span></h2>

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    metric_names=['validation:accuracy'],
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_df = lineage_table.dataframe()
lineage_df.shape

In [ ]:
lineage_df

# Prediction 수행

###  Raw Text를 BERT Tokens로 변환하기 위한 Request Handler 설정

In [ ]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)

###  BERT Response를 Predicted Classes로 변환하기 위한 Response Handler 설정

In [ ]:
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes

In [ ]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[1, 2, 3, 4, 5])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

In [ ]:
import tensorflow as tf
import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

In [ ]:
%store endpoint_name

# Using API Gateway with SageMaker Endpoints

https://aws.amazon.com/blogs/machine-learning/creating-a-machine-learning-powered-rest-api-with-amazon-api-gateway-mapping-templates-and-amazon-sagemaker/

# Delete Endpoint

In [ ]:
client = boto3.client('sagemaker')

client.delete_endpoint(
    EndpointName=endpoint_name
)